In [67]:
def ohe(train):
    import pandas as pd
    original_columns = train.columns.tolist()
    train_encoded = pd.get_dummies(train, columns=['ACCOUNT_TYPE','OWNERSHIP_TYPE','OCCUPATION_TYPE'], drop_first=True)
    train.drop(columns=['ACCOUNT_TYPE'], inplace=True)
    train = train_encoded
    print(train.columns)
    
    return train


In [68]:
import pandas as pd
df_train=pd.read_csv("feature_generated.csv")
df_train = ohe(df_train)
df=pd.read_csv("train_main_loan.csv")

Index(['ID', 'HIGH_CREDIT_OR_SANCTIONED_AMOUNT', 'MONTH_OPENED',
       'CURRENT_BALANCE', 'ACTUAL_PAYMT_AMT', 'REPAYMENT_TENURE',
       'AMOUNT_OVERDUE', 'PAYMENT_HISTORY_1', 'COLLATERALVALUE', 'TU_SCORE',
       'MONTH_PAYMENT_HISTORY_START', 'MONTH_PAYMENT_HISTORY_END',
       'MONTH_CERTIFIED', 'MONTH_LAST_PAYMENT', 'AGE', 'GENDER', 'ACTUAL_ROI',
       'COUNT_PAID', 'COUNT_LATE', 'COUNT_MORE_THAN_90', 'COUNT_0_TO_60',
       'AVG_DAYS', 'COUNT_MISSING_DAYS', 'PERSONAL_LOANS', 'PROPERTY_LOANS',
       'BUSINESS_LOANS', 'GOLD_LOANS', 'HOUSING_LOANS',
       'KISAN_CREDIT_CARD_LOANS', 'EDUCATION_LOANS', 'CLOSED_PERSONAL_LOANS',
       'CLOSED_PROPERTY_LOANS', 'CLOSED_BUSINESS_LOANS', 'CLOSED_GOLD_LOANS',
       'CLOSED_HOUSING_LOANS', 'CLOSED_KISAN_CREDIT_CARD_LOANS',
       'CLOSED_EDUCATION_LOANS', 'PROPERTY_LOANS_PAID_SUM',
       'BUSINESS_LOANS_PAID_SUM', 'GOLD_LOANS_PAID_SUM',
       'HOUSING_LOANS_PAID_SUM', 'PERSONAL_LOANS_PAID_SUM',
       'KISAN_CREDIT_CARD_LOANS_PAID_SUM'

In [69]:
df = df[df['OCCUPATION_TYPE'] != 'R18']

### ENGINEERED FEATURES

In [70]:
import numpy as np
df_train['new_feature-1'] = ((df_train['CURRENT_BALANCE'])+(df_train['ACTUAL_PAYMT_AMT']))
df_train['new_feature-2'] = ((df_train['MONTH_OPENED'])+(df_train['REPAYMENT_TENURE']))
df_train['EMI_AMOUNT']=df['EMI_AMOUNT']
df_train['EMI_AMOUNT'] = df_train['EMI_AMOUNT'].fillna(df_train['ACTUAL_PAYMT_AMT'])

In [71]:
selected_features = ['HIGH_CREDIT_OR_SANCTIONED_AMOUNT', 'ACTUAL_PAYMT_AMT', 'REPAYMENT_TENURE', 'ACCOUNT_TYPE_Housing Loan',
       'ACCOUNT_TYPE_Personal Loan', 'ACCOUNT_TYPE_Property Loan',
       'OWNERSHIP_TYPE_Joint', 'OCCUPATION_TYPE_SALARIED',
       'OCCUPATION_TYPE_SENP', 'OCCUPATION_TYPE_SEP','new_feature-1','new_feature-2','EMI_AMOUNT']

X = df_train[selected_features]
y = df_train['ACTUAL_ROI']

In [72]:
# Assuming X is your DataFrame
nan_columns = X.columns[X.isna().any()].tolist()

if nan_columns:
    print("Features with NaN values:", nan_columns)
else:
    print("No features have NaN values in X.")


No features have NaN values in X.


In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)

# Initialize regression models
regressors = {
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'LGBM': LGBMRegressor(),  # Use LGBM instead of AdaBoost
}

# Train and evaluate each model without scaling
for name, model in regressors.items():
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    train_r2 = r2_score(y_train, train_pred)

    # Calculate RMSE and R2 for the testing set
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    test_r2 = r2_score(y_test, test_pred)
    
    print(f'{name}:')
    print(f'Training R2: {train_r2:.4f}')
    print(f'Training RMSE: {train_rmse:.4f}')
    print(f'Testing R2: {test_r2:.4f}')
    print(f'Testing RMSE: {test_rmse:.4f}')
    print('------------------------')


Random Forest:
Training R2: 0.9817
Training RMSE: 0.9436
Testing R2: 0.8979
Testing RMSE: 2.2362
------------------------
XGBoost:
Training R2: 0.9159
Training RMSE: 2.0243
Testing R2: 0.8785
Testing RMSE: 2.4394
------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000647 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1544
[LightGBM] [Info] Number of data points in the train set: 55965, number of used features: 13
[LightGBM] [Info] Start training from score 16.000578
LGBM:
Training R2: 0.8831
Training RMSE: 2.3860
Testing R2: 0.8687
Testing RMSE: 2.5357
------------------------


In [74]:
X

,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,ACTUAL_PAYMT_AMT,REPAYMENT_TENURE,ACCOUNT_TYPE_Housing Loan,ACCOUNT_TYPE_Personal Loan,ACCOUNT_TYPE_Property Loan,OWNERSHIP_TYPE_Joint,OCCUPATION_TYPE_SALARIED,OCCUPATION_TYPE_SENP,OCCUPATION_TYPE_SEP,new_feature-1,new_feature-2,EMI_AMOUNT
0,818517,8627.0,142.0,True,False,False,True,True,False,False,649369.0,257.0,6106.0
1,1243755,500.0,156.0,True,False,False,True,True,False,False,949946.0,289.0,11445.0
2,1826422,16036.0,148.0,True,False,False,True,True,False,False,1312885.0,229.0,16036.0
3,1847916,26996.0,174.0,True,False,False,True,True,False,False,1751252.0,185.0,26996.0
4,2318386,25131.0,116.0,True,False,False,True,True,False,False,1816068.0,226.0,25131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69952,2600000,30762.0,162.0,False,False,True,True,False,True,False,2412199.0,231.0,30762.0
69953,1016188,29851.0,48.0,False,False,False,True,False,True,False,820337.0,63.0,29851.0
69954,335475,10388.0,48.0,False,False,False,True,False,True,False,291494.0,68.0,10388.0
69955,1585160,16739.0,143.0,True,False,False,True,True,False,False,1271974.0,243.0,16739.0


In [75]:
df=pd.read_csv('missing_handled_main_test.csv')
df_test=pd.read_csv('test_main_loan.csv')

In [76]:
df

,ID,ACCOUNT_TYPE,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,MONTH_OPENED,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,REPAYMENT_TENURE,AMOUNT_OVERDUE,PAYMENT_HISTORY_1,OWNERSHIP_TYPE,...,MONTH_LAST_PAYMENT,AGE,OCCUPATION_TYPE,GENDER,COUNT_PAID,COUNT_LATE,COUNT_MORE_THAN_90,COUNT_0_TO_60,AVG_DAYS,COUNT_MISSING_DAYS
0,A000614177,Housing Loan,2277986,91,1824507,22985.0,149.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,35,SEP,Male,17.0,0.0,0.0,0.0,0.0,1.0
1,A001744326,Business Loan,1030000,14,819209,30256.0,48.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,47,SENP,Male,18.0,0.0,0.0,0.0,0.0,0.0
2,A000242802,Housing Loan,960705,3,908604,25229.0,175.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,59,SENP,Female,18.0,0.0,0.0,0.0,0.0,0.0
3,B1000244547,Housing Loan,1437990,69,1367609,13067.0,259.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,54,SENP,Male,17.0,0.0,0.0,0.0,0.0,1.0
4,B3000178046,Business Loan,728474,1,718224,22350.0,48.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,50,SENP,Male,18.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29978,A002307275,Housing Loan,949186,66,929479,8890.0,300.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,39,SALARIED,Male,17.0,0.0,0.0,0.0,0.0,1.0
29979,A000079323,Business Loan,765335,15,517748,28054.0,36.0,0.0,000000000000000000000000000000000000XXX0000000...,Joint,...,0.0,43,SALARIED,Female,17.0,0.0,0.0,0.0,0.0,1.0
29980,A002385848,Housing Loan,920928,105,916673,9974.0,374.0,0.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,46,SENP,Male,17.0,0.0,0.0,0.0,0.0,1.0
29981,A002435483,Housing Loan,1108527,0,1108527,10472.0,175.0,2733.0,0000000000000000000000000000000000000000000000...,Joint,...,0.0,38,SALARIED,Female,17.0,1.0,0.0,1.0,21.0,0.0


In [77]:
import numpy as np
df['new_feature-1'] = ((df['CURRENT_BALANCE'])+(df['ACTUAL_PAYMT_AMT']))
df['new_feature-2'] = ((df['MONTH_OPENED'])+(df['REPAYMENT_TENURE']))
df['EMI_AMOUNT']=df_test['EMI_AMOUNT']
df['EMI_AMOUNT'] = df['EMI_AMOUNT'].fillna(df['ACTUAL_PAYMT_AMT'])

In [78]:
df = ohe(df)


Index(['ID', 'HIGH_CREDIT_OR_SANCTIONED_AMOUNT', 'MONTH_OPENED',
       'CURRENT_BALANCE', 'ACTUAL_PAYMT_AMT', 'REPAYMENT_TENURE',
       'AMOUNT_OVERDUE', 'PAYMENT_HISTORY_1', 'COLLATERALVALUE', 'TU_SCORE',
       'MONTH_PAYMENT_HISTORY_START', 'MONTH_PAYMENT_HISTORY_END',
       'MONTH_CERTIFIED', 'MONTH_LAST_PAYMENT', 'AGE', 'GENDER', 'COUNT_PAID',
       'COUNT_LATE', 'COUNT_MORE_THAN_90', 'COUNT_0_TO_60', 'AVG_DAYS',
       'COUNT_MISSING_DAYS', 'new_feature-1', 'new_feature-2', 'EMI_AMOUNT',
       'ACCOUNT_TYPE_Housing Loan', 'ACCOUNT_TYPE_Personal Loan',
       'ACCOUNT_TYPE_Property Loan', 'OWNERSHIP_TYPE_Joint',
       'OCCUPATION_TYPE_SALARIED', 'OCCUPATION_TYPE_SENP',
       'OCCUPATION_TYPE_SEP', 'OCCUPATION_TYPE_Salaried'],
      dtype='object')


In [79]:
selected_features = ['HIGH_CREDIT_OR_SANCTIONED_AMOUNT', 'ACTUAL_PAYMT_AMT', 'REPAYMENT_TENURE', 'ACCOUNT_TYPE_Housing Loan',
       'ACCOUNT_TYPE_Personal Loan', 'ACCOUNT_TYPE_Property Loan',
       'OWNERSHIP_TYPE_Joint', 'OCCUPATION_TYPE_SALARIED',
       'OCCUPATION_TYPE_SENP', 'OCCUPATION_TYPE_SEP','new_feature-1','new_feature-2','EMI_AMOUNT','ID']
X_new = df[selected_features]


In [80]:
X_new

,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,ACTUAL_PAYMT_AMT,REPAYMENT_TENURE,ACCOUNT_TYPE_Housing Loan,ACCOUNT_TYPE_Personal Loan,ACCOUNT_TYPE_Property Loan,OWNERSHIP_TYPE_Joint,OCCUPATION_TYPE_SALARIED,OCCUPATION_TYPE_SENP,OCCUPATION_TYPE_SEP,new_feature-1,new_feature-2,EMI_AMOUNT,ID
0,2277986,22985.0,149.0,True,False,False,True,False,False,True,1847492.0,240.0,22985.0,A000614177
1,1030000,30256.0,48.0,False,False,False,True,False,True,False,849465.0,62.0,30256.0,A001744326
2,960705,25229.0,175.0,True,False,False,True,False,True,False,933833.0,178.0,25229.0,A000242802
3,1437990,13067.0,259.0,True,False,False,True,False,True,False,1380676.0,328.0,13067.0,B1000244547
4,728474,22350.0,48.0,False,False,False,True,False,True,False,740574.0,49.0,21778.0,B3000178046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29978,949186,8890.0,300.0,True,False,False,True,True,False,False,938369.0,366.0,8890.0,A002307275
29979,765335,28054.0,36.0,False,False,False,True,True,False,False,545802.0,51.0,28054.0,A000079323
29980,920928,9974.0,374.0,True,False,False,True,False,True,False,926647.0,479.0,9974.0,A002385848
29981,1108527,10472.0,175.0,True,False,False,True,True,False,False,1118999.0,175.0,10472.0,A002435483


In [81]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have X and y defined earlier for training
# Assuming you have X_to_predict defined earlier for prediction

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)

# Initialize the Random Forest model
random_forest_model = RandomForestRegressor()

# Fit the Random Forest model on the training data
random_forest_model.fit(X_train, y_train)
ids = X_new['ID']
X_new.drop(columns='ID',inplace=True)

C:\Users\risha\AppData\Local\Temp\ipykernel_22780\724562580.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new.drop(columns='ID',inplace=True)


In [82]:

# Make predictions on the 'X_to_predict' dataset
predictions = random_forest_model.predict(X_new)

# Assuming you have a column 'ID' in your original data

# Create a DataFrame with 'ID' and 'ACTUAL_ROI' columns
results_df = pd.DataFrame({'ID': ids, 'ACTUAL_ROI': predictions})

# For demonstration, you can print the final DataFrame
print(results_df)

# Save the final DataFrame to a CSV file if needed
results_df.to_csv('random_forest_predictions.csv', index=False)


                ID  ACTUAL_ROI
0       A000614177     11.3799
1       A001744326     18.0400
2       A000242802     10.8113
3      B1000244547     11.3884
4      B3000178046     20.6908
...            ...         ...
29978   A002307275     11.0435
29979   A000079323     20.5164
29980   A002385848     13.4764
29981   A002435483     10.4200
29982   A001881482     11.3659

[29983 rows x 2 columns]


In [25]:
X

,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,ACTUAL_PAYMT_AMT,REPAYMENT_TENURE,ACCOUNT_TYPE_Housing Loan,ACCOUNT_TYPE_Personal Loan,ACCOUNT_TYPE_Property Loan,OWNERSHIP_TYPE_Joint,OCCUPATION_TYPE_SALARIED,OCCUPATION_TYPE_SENP,OCCUPATION_TYPE_SEP,new_feature-1,new_feature-2,EMI_AMOUNT
0,818517,8627.0,142.0,True,False,False,True,True,False,False,649369.0,257.0,6106.0
1,1243755,500.0,156.0,True,False,False,True,True,False,False,949946.0,289.0,11445.0
2,1826422,16036.0,148.0,True,False,False,True,True,False,False,1312885.0,229.0,16036.0
3,1847916,26996.0,174.0,True,False,False,True,True,False,False,1751252.0,185.0,26996.0
4,2318386,25131.0,116.0,True,False,False,True,True,False,False,1816068.0,226.0,25131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69952,2600000,30762.0,162.0,False,False,True,True,False,True,False,2412199.0,231.0,30762.0
69953,1016188,29851.0,48.0,False,False,False,True,False,True,False,820337.0,63.0,29851.0
69954,335475,10388.0,48.0,False,False,False,True,False,True,False,291494.0,68.0,10388.0
69955,1585160,16739.0,143.0,True,False,False,True,True,False,False,1271974.0,243.0,16739.0


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assume you have X and y defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)

# Define the random forest regressor
rf_regressor = RandomForestRegressor()

# Set up parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
}

# Use GridSearchCV for tuning with verbose and scoring set to 'r2'
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=5, scoring='r2', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Initialize random forest with the best parameters
final_rf_model = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                       max_features=best_params['max_features'])

# Train and evaluate the final random forest model
final_rf_model.fit(X_train, y_train)
train_pred = final_rf_model.predict(X_train)
test_pred = final_rf_model.predict(X_test)

# Calculate metrics for training set
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
train_r2 = r2_score(y_train, train_pred)

# Calculate metrics for testing set
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
test_r2 = r2_score(y_test, test_pred)

# Print results
print('Random Forest:')
print(f'Best Parameters: {best_params}')
print(f'Training R2: {train_r2:.4f}')
print(f'Training RMSE: {train_rmse:.4f}')
print(f'Testing R2: {test_r2:.4f}')
print(f'Testing RMSE: {test_rmse:.4f}')
print('------------------------')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END .................max_features=auto, n_estimators=50; total time=   0.0s
[CV] END .................max_features=auto, n_estimators=50; total time=   0.0s
[CV] END .................max_features=auto, n_estimators=50; total time=   0.0s
[CV] END .................max_features=auto, n_estimators=50; total time=   0.0s
[CV] END .................max_features=auto, n_estimators=50; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=100; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=100; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=100; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=100; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=100; total time=   0.0s
[CV] END ................max_features=auto, n_estimators=200; total time=   0.0s
[CV] END ................max_features=auto, n_es

c:\Users\risha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\risha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\risha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\risha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py

KeyboardInterrupt: 

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Assuming X and y are defined somewhere in your code

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a neural network model with 2 additional layers, dropout, and early stopping
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(1, activation='linear'))

# Compile the model with early stopping
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Make predictions
train_pred = model.predict(X_train_scaled)
test_pred = model.predict(X_test_scaled)

# Evaluate the model
train_mae = mean_absolute_error(y_train, train_pred)
test_mae = mean_absolute_error(y_test, test_pred)

print(f'Training MAE: {train_mae:.4f}')
print(f'Testing MAE: {test_mae:.4f}')


Epoch 1/30
1749/1749 [==============================] - 7s 4ms/step - loss: 120.6884 - val_loss: 43.9232
Epoch 2/30
1749/1749 [==============================] - 6s 3ms/step - loss: 70.7386 - val_loss: 41.6188
Epoch 3/30
1749/1749 [==============================] - 6s 3ms/step - loss: 60.4750 - val_loss: 32.8834
Epoch 4/30
1749/1749 [==============================] - 6s 3ms/step - loss: 51.9687 - val_loss: 31.0308
Epoch 5/30
1749/1749 [==============================] - 6s 3ms/step - loss: 43.8316 - val_loss: 27.2400
Epoch 6/30
1749/1749 [==============================] - 6s 4ms/step - loss: 39.4529 - val_loss: 23.6754
Epoch 7/30
1749/1749 [==============================] - 6s 3ms/step - loss: 35.3834 - val_loss: 22.3598
Epoch 8/30
1749/1749 [==============================] - 6s 3ms/step - loss: 32.8654 - val_loss: 24.0901
Epoch 9/30
1749/1749 [==============================] - 6s 3ms/step - loss: 30.7569 - val_loss: 24.9605
Epoch 10/30
1749/1749 [==============================] - 6s 4ms